In [3]:

import sys
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import os
from os.path import isfile,join
import xml.etree.cElementTree as ET
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import time
import re
import pickle
#import doc2vec_impl
import gensim
import nltk
import os
nltk.download('stopwords')
nltk.download('punkt')
from PyRouge.pyrouge import Rouge

import FileProcess as fp


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


('Found embeddings: ', 31105, '/', 42514)
('word_index len = ', 41522)
('time-taken for FP: ', 3.247076988220215)


In [64]:
print (fp.window_size)
batch_size=10
vocabulary_size=400000
embedding_size=50
learning_rate = 0.1
#embedding_filename = "glove_6B_200d.txt"
embedding_filename = "word2vec.model"   ##Trying with pre-trained word2vec model
training_epochs=1
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
k = 20

2


In [5]:
print(fp.word_index.get('considered'))
print (fp.word_index)
# from gensim.test.utils import common_texts, get_tmpfile
# from gensim.models import Word2Vec
# import gensim.models.keyedvectors as word2vec
# embed_map = word2vec.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
# print ('kandy')

23131
{'fawn': 40806, '': 42513, '999755': 25315, 'deferment': 12529, '11543': 21992, 'woods': 36688, 'spiders': 18633, 'hanging': 19424, 'woody': 9757, 'trawling': 4284, 'localized': 35820, 'regularize': 3913, 'disobeying': 25601, '5988': 5245, 'canes': 36129, '5982': 2688, 'fractioning': 6012, '5980': 25046, '5984': 41804, 'originality': 21744, 'refunding': 37134, 'caned': 37481, 'alphabetic': 12170, 'appointers': 14079, 'wracked': 17650, '493654': 19048, 'stipulate': 16640, 'pigment': 18922, 'appropriation': 15934, 'medicament': 18286, 'uncoordinated': 7325, 'showroom': 25556, 'advices': 38361, 'wooded': 28328, 'liaisons': 25877, 'broiler': 11420, 'wooden': 7428, 'wholemeal': 9566, 'viable': 25396, 'abatements': 37751, '11541': 24273, 'amplifications': 34892, 'deferments': 4674, 'busybody': 35303, 'immunities': 29866, '034207092': 30302, '0058': 39230, 'sooty': 30377, 'insular': 42030, '0053': 14249, '0051': 42092, 'scrapes': 21164, 'scraper': 2338, '270': 17246, '271': 19325, '272'

In [6]:
dirpath = "corpus/fulltext/"
files = [f for f in os.listdir(dirpath) if isfile(join(dirpath, f))]

files_2006=[];
files_2007=[];
files_2008=[];
files_2009=[];

for file in files:
    if file.split('_')[0]=='06':
        files_2006.append(file);
    if file.split('_')[0]=='07':
        files_2007.append(file);
    if file.split('_')[0]=='08':
        files_2008.append(file);
    if file.split('_')[0]=='09':
        files_2009.append(file);

#print(fp.get_sentences(files[0]))

In [7]:
stop_words_list=set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

##########loading dev2vec=============
#doc2vec_impl.main();

d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec_models_50len/doc2vec.model')
docvec = d2v_model.docvecs[files[0]]
print(docvec.shape)

(50,)


In [8]:
#
def read_data(file_name):
    with open(file_name, 'r') as f:
        word_vocab = set()  # not using list to avoid duplicate entry
        word_count = {};
        word_index={};

        word2vector = {}
        count=0;
        for line in f:
            line_ = line.strip()  # Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word_index[words_Vec[0]]=count;
            count=count+1;

            word2vector[words_Vec[0]] = np.array(words_Vec[1:], dtype=float)
    print("Total Words in DataSet:", len(word_vocab))
    return word_vocab, word2vector, word_index

In [9]:
def load_embeddings(filename):
    dvocab, w2v,word_index  = read_data(filename);
    return w2v,word_index

In [10]:
embedding_array = fp.embedding_array
word_index = fp.word_index


reverseWordIndex = {}
for key in word_index:
    reverseWordIndex[word_index[key]] = key

In [ ]:
# ## reducing word embeddings to 50
# if embedding_size==50:
#     rand = np.random.rand(200,embedding_size)
#     embedding_array_50={} 
#     for key in embedding_array:
#         embedding_array_50[key] = np.matmul(embedding_array[key].reshape(1, 200), rand)
#     print (embedding_array_50['biennials'].shape)

In [11]:
#### TO avoid downloading tf hub modules everytime
##  export TFHUB_CACHE_DIR=/usr/local/bin

#sen2vec embeddings
def sen2vec1(sentences):
    embed = hub.Module(module_url)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        sentenceEmbeddings = session.run(embed(sentences))    
        return sentenceEmbeddings

    
def sent2vector(sent):
    words = word_tokenize(sent.lower())
    emb=[]
    for w in words:
        if w in embedding_array.keys():
            emb.append(embedding_array.get(w));
        else:
            emb.append(embedding_array.get('unk'));
    return np.mean(np.array(emb),axis=0)

def get_sent_embeddings(sent_list):
    out_vec=np.zeros((len(sent_list)-1,200))
    for i  in range(len(sent_list)-1):
        out_vec[i]=sent2vector(sent_list[i]);
    return out_vec
    
# sentence_embeddings = sen2Vec(['UWA was ordered to pay the costs of Dr Gra','Sirtex succeeded in its cross-claim against Dr'])

In [12]:
def get_statements(file):
    #with open(dirpath + file, 'r',encoding="utf-8", errors='replace') as f:
    with open(dirpath + file, 'r') as f:
        data=str(f.read());
        data=data.lower()
        data = data.replace("\"id=", "id=\"");
        data=data.replace("\n","")
        data=data.replace('".*?=.*?"', "",)
        data=data.replace("&","");
        xml = ET.fromstring(str(data))
        name=None;
        rows_list=[];
        catchphrases=[];
        sentences=[];
        for child in xml:
            if child.tag=="catchphrases":
                for catchphrase in child:
                    id=catchphrase.attrib.get("id")
                    #print(catchphrase.text)
                    catchphrases.append({"file_id":file,"Name":name,"Id":id,"text":catchphrase.text})
                    #catchphrases+=tokenizer.tokenize(catchphrase.text)
            if child.tag=="sentences":
                for sentence in child:
                    id = sentence.attrib.get("id")
                    sentences.append({"file_id":file,"Name":name,"Id":id,"text":sentence.text})
                    #sentences+=tokenizer.tokenize(sentence.text)
    
    
    #sentences = [w for w in sentences if not w in stop_words_list] 
    #catchphrases = [w for w in catchphrases if not w in stop_words_list]
    
    
    #sentences=[word.lower() for word in sentences if word.isalphanum()]
    #catchphrases=[word.lower() for word in catchphrases if word.isalphanum()]
    
    return sentences,catchphrases

In [13]:
sentences,catchphrases=get_statements(files[0])
start=time.time()
sentences, catch_words = fp.get_dataframe(pd.DataFrame(sentences),pd.DataFrame(catchphrases))
print(sentences,catch_words)
print("time-taken:",time.time()-start)

(        Id     file_id  is_catchword                                words
0       s0  07_209.xml             0      [9205, 3424, 27288, 9330, 9272]
1       s0  07_209.xml             1     [3424, 27288, 9330, 9272, 13945]
2       s0  07_209.xml             0    [27288, 9330, 9272, 13945, 24684]
3       s0  07_209.xml             0     [9330, 9272, 13945, 24684, 1109]
4       s0  07_209.xml             0    [9272, 13945, 24684, 1109, 27288]
5       s0  07_209.xml             0   [13945, 24684, 1109, 27288, 34485]
6       s0  07_209.xml             0   [24684, 1109, 27288, 34485, 39252]
7       s0  07_209.xml             0    [1109, 27288, 34485, 39252, 1109]
8       s0  07_209.xml             0   [27288, 34485, 39252, 1109, 40642]
9       s0  07_209.xml             0   [34485, 39252, 1109, 40642, 42512]
10      s1  07_209.xml             0      [812, 9467, 2100, 42512, 30971]
11      s1  07_209.xml             0    [9467, 2100, 42512, 30971, 23350]
12      s1  07_209.xml             0 

In [14]:
def prepare_index_matrices(sentence_indexes):
    length=len(sentence_indexes);
    
    sentence_indexes=[word_index['unk'],word_index['unk']]+sentence_indexes+[word_index['unk'],word_index['unk']];    
    out_indices=[]
    for i in range(2,length):
        out_indices.append(sentence_indexes[i-2:i+3]);
        
    return out_indices;

def prepare_label_matrices(labels):
    length=len(labels);
    labels=[0,0]+labels+[0,0];
    out_labels=[];
    
    for i in range(2,length):
        out_labels.append(labels[i]);
        
    return out_labels;
    

In [15]:
def lookup_indexes_labels(sentences,catchphrases):
    
    sentence_indexes=[];
    labels=[];
    
    for word in sentences:
        if word in word_index:
            sentence_indexes.append(word_index[word]);
        else:
            #print("unknown word",word)
            sentence_indexes.append(word_index['unk']);
            
    for word in sentences:
        if word in catchphrases:
            labels.append(1.0);
        else:
            #print("unknown word",word)
            labels.append(0.0);
            
    return sentence_indexes,labels;

In [16]:
def get_file_sentences(file_name):
    temp_data=full_data[(full_data.file_id==file_name) & full_data['Id'].str.startswith(('s'))]
    return list(temp_data.text.values)[:-1]

In [25]:
rouge = Rouge()
def get_phrases_scores(true_list, predicted_list):
    from collections import defaultdict
    scores_map = defaultdict(float)
    for phrase in predicted_list:
        max_prec=0;
        for true_phrase in true_list:
            [precision, recall, f_score] = rouge.rouge_l(phrase, true_phrase);
            if precision>max_prec:
                max_prec=precision;
        scores_map[phrase]=max_prec;
        #print(max_prec)
    #print(scores);
    #return np.mean(np.array(scores));
    return scores_map

In [20]:
import time

class CatchPhraseExtractor(object):

    def __init__(self, graph, embedding_array_1):
        self.build_graph(graph, embedding_array_1)

    def build_graph(self, graph, embedding_array_2):
        print("generating graph");

        with graph.as_default():            
            self.phrases_indices = tf.placeholder(tf.int32, shape=[None, 5])
            self.phrase_labels = tf.placeholder(tf.float32, shape=[None, 1])
            
            self.test_phrases_indices = tf.placeholder(tf.int32, shape=[None, 5])
            
            self.W = tf.Variable(embedding_array_2, dtype=tf.float32)
            #W = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            phrase_tensor = tf.nn.embedding_lookup(self.W, self.phrases_indices)
            
            #self.sentence_indices = tf.placeholder(tf.int32, shape=[None,1])
            #self.sentence_embeddings = tf.placeholder(tf.float32, shape=[None,200])
            #sentence_tensor = tf.nn.embedding_lookup(self.sentence_embeddings, self.sentence_indices)
            #sentence_tensor=tf.reshape(sentence_tensor, [tf.shape(sentence_tensor)[0], 200])
                                                     
            #self.doc_embedding = tf.placeholder(tf.float32, shape=[1,embedding_size])
            #document_tensor = tf.tile(self.doc_embedding, [tf.shape(self.phrases_indices)[0], 1])
            
            phrase_tensor = tf.reshape(phrase_tensor, [tf.shape(phrase_tensor)[0], embedding_size*5])
            conv_filter = tf.Variable(tf.truncated_normal((embedding_size*5, embedding_size), stddev=0.1))
            word_tensor = tf.nn.relu(tf.matmul(phrase_tensor, conv_filter))
            
            #stacked_tensor = tf.concat(values=[word_tensor, sentence_tensor, document_tensor], axis=1)
            #stacked_tensor = tf.concat(values=[word_tensor, document_tensor], axis=1)
            stacked_tensor = tf.concat(values=[word_tensor], axis=1)
            weights_input = tf.Variable(tf.random_normal((embedding_size*1, 100), stddev=0.2))
            weights_input_2 = tf.Variable(tf.random_normal((100, 1), stddev=0.2))
            
            bias1 = tf.Variable(tf.random_normal((1, 100), stddev=0.2))
            bias2 = tf.Variable(tf.random_normal((1, 1), stddev=0.2))
            
            self.sentence_values=self.MLP(stacked_tensor, weights_input, weights_input_2, bias1, bias2)
            
            #prediction_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.phrase_labels,logits=self.sentence_values)
            #self.loss = tf.reduce_mean(prediction_loss)
            self.loss=tf.reduce_mean(tf.keras.metrics.mean_squared_error(self.phrase_labels, self.sentence_values))
            #self.loss=tf.convert_to_tensor(tf.reduce_mean(self.sentence_values));
            
            #using the gradient descent optimizer
            optimizer = tf.train.GradientDescentOptimizer(learning_rate)
            grads = optimizer.compute_gradients(self.loss)
            #clipped_grads = [(tf.clip_by_norm(grad, 5), var) for grad, var in grads]
            self.app = optimizer.apply_gradients(grads)
            
            #For test data
            test_embed = tf.nn.embedding_lookup(self.W, self.test_phrases_indices)
            test_embed = tf.reshape(test_embed, [tf.shape(test_embed)[0], embedding_size*5])
            test_conv_filter = tf.Variable(tf.truncated_normal((embedding_size*5, embedding_size), stddev=0.1));
            test_word_tensor = tf.nn.relu(tf.matmul(test_embed, test_conv_filter));
            self.test_pred = self.MLP(test_word_tensor, weights_input, weights_input_2, bias1, bias2)
            
            self.init = tf.global_variables_initializer()
            
    def MLP(self, stacked_tensor, weights_input, weights_input_2, bias1, bias2):
        h1 = (tf.matmul(weights_input,stacked_tensor,transpose_a=True,transpose_b=True))
        #h1 = tf.add(h1, bias1)
        h1 = tf.nn.tanh(h1)
        h2 = tf.matmul(weights_input_2,h1,transpose_a=True,transpose_b=False)
        #h2 = tf.add(h2, bias2)
        h2 = tf.nn.sigmoid(h2)
        #h2 = tf.round(tf.transpose(h2))
        return tf.transpose(h2)
        #return tf.cast(tf.to_float(h2>0.5),dtype=tf.float32)

    '''
    def gradient_descent(self,total_loss):
        total_loss=tf.convert_to_tensor(total_loss)
        with tf.GradientTape(persistent=True) as tape:
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
            print(variables)
        return total_loss, tape.gradient(tf.convert_to_tensor(total_loss), variables)
    '''
    
    def train(self, sess, num_steps):
        self.init.run()
        #print("Initailized")
        start=time.time()
        total_loss=0.0;
        train_files = files_2006 + files_2007 + files_2008
        for file in train_files:
            start=time.time()
            file_data_frame = None; catch_words = None
            try:
                sentences, catchphrases=get_statements(file);
                file_data_frame, catch_words = fp.get_dataframe(pd.DataFrame(sentences),pd.DataFrame(catchphrases))
            except:
                print('************************BAD DATA IN FILE, file=', file)
                continue
            #print("Time taken:",time.time()-start)
            #needs sentences list
            #sentences_list = fp.get_sentences(file) #indexes/order preserved
            #sentence_embeddings = sen2vec(sentences_list)
            #sentence_embeddings = get_sent_embeddings(sentences_list)
            
            #sentence_ids_to_indices = np.array([int(x[1:]) for x in list(file_data_frame['Id'].values)])
            
            #phrase inputs and labels
            phrases_indices = np.array(list(file_data_frame['words'].values))
            phrases_label_matrix = np.array(list(file_data_frame['is_catchword'].values))
            
            
            #doc embedding
            #doc_embedding = d2v_model.docvecs[file]

            
            #doc_embedding = np.reshape(doc_embedding, (1, embedding_size))
            #sentence_ids_to_indices = np.reshape(sentence_ids_to_indices, (sentence_ids_to_indices.shape[0], 1))
            phrases_label_matrix = np.reshape(phrases_label_matrix, (phrases_label_matrix.shape[0], 1))
            
            '''
            print ('*************Doc: ', doc_embedding.shape)
            print ('*************phrases_indices: ', phrases_indices.shape)
            print ('*************sentence_ids_to_indices: ', sentence_ids_to_indices.shape)
            print ('*************phrases_label_matrix: ', phrases_label_matrix.shape)
            print ('*************sentence_embeddings: ', sentence_embeddings.shape)           
            
            feed_dict = {self.doc_embedding: doc_embedding, 
                         self.phrases_indices: phrases_indices, 
                         self.phrase_labels: phrases_label_matrix, 
                         self.sentence_indices:sentence_ids_to_indices, 
                         self.sentence_embeddings: sentence_embeddings}'''
            
            feed_dict = {#self.doc_embedding: doc_embedding, 
                         self.phrases_indices: phrases_indices, 
                         self.phrase_labels: phrases_label_matrix}
            
            #print("sending inputs");
            #loss_val = sess.run([self.loss], feed_dict=feed_dict)  
            _,loss_val,outputs = sess.run([self.app,self.loss,self.sentence_values], feed_dict=feed_dict)
            
            

            if num_steps%20 == 0:
                print('Train Finished for file ', file, ' loss_val=', loss_val)
            #for i in range(len(outputs)):
            #    print(outputs[i])
       
    def get_phrases(self, phrase_indices, output):
        catch_phrases = []
        #print ('type', type(output[0].tolist()))
        #print (', size: ', len(output[0].tolist()))
        #import pdb; pdb.set_trace()
        #output = output[0].tolist()
        for i, out in enumerate(output):
            if out == 1:
                phrase = ''
                for ind in range(5):
                    phrase += reverseWordIndex[phrase_indices[i][ind]] + " "
                catch_phrases.append(phrase[:-1])
        return catch_phrases
            

    def test(self, sess, file):
        try:
            sentences, catchphrases = get_statements(file);
            file_data_frame, catch_words = fp.get_dataframe(pd.DataFrame(sentences),pd.DataFrame(catchphrases))
        except:
            print('************************BAD DATA IN FILE, file=', file)
            return {}
        
        test_phrases_indices = np.array(list(file_data_frame['words'].values))
        phrases_label_matrix = np.array(list(file_data_frame['is_catchword'].values))
        phrases_label_matrix = np.reshape(phrases_label_matrix, (phrases_label_matrix.shape[0], 1))
            
        feed_dict = {self.test_phrases_indices: test_phrases_indices}
        outputs = sess.run([self.test_pred], feed_dict=feed_dict)
        
        outputs = [1 if x[0]>=0.5 else 0 for x in outputs[0].tolist()] 

        predicted_catch_phrases = self.get_phrases(test_phrases_indices, outputs)
        #print ('In test', 'file=',file, ', catchphrases=',catchphrases, 'predicted_catch_phrases=',predicted_catch_phrases)
        #print ('Outputs=', outputs, ' test_phrase_indices=',test_phrases_indices)
        return get_phrases_scores(catchphrases, predicted_catch_phrases)
        
        
    def load_embeddings(self,embedding_array):
        sess.run(self.embedding_init, feed_dict={self.embedding_placeholder: np.asarray(list(embedding_array.values()))})
        

    

In [21]:
def get_features(inputs):
    out=np.zeros((len(inputs),embedding_size));
    for i in range(len(inputs)):
        try:
            c = w2v[inputs[i]]
        except KeyError:
            c = np.zeros((1,embedding_size));
        out[i]=c;
    
    return np.transpose(out);

In [22]:
graph = tf.Graph()
model = CatchPhraseExtractor(graph, embedding_array)


generating graph


In [ ]:
'''test_scores = {}

with tf.Session(graph=graph) as sess:
    start = time.time()
    for i in range(1):
        model.train(sess, i)
    t=time.time()-start
    print ('**************************Train_Time = epoch', t, '  mins: ',divmod(divmod(t, 3600)[1], 60)[0])
    for file in files[-200:]:
        print('Testing for ', file) 
        score = model.test(sess, file)
        test_scores[file] = score
        
    #optimizer.apply_gradients(zip(grads, model.variables), global_step)

print ('***************** DONE *******************')
print (test_scores)
'''

In [23]:
## Training model

test_scores = {}

sess = tf.InteractiveSession(graph=graph)
start = time.time()
for i in range(1):
    model.train(sess, i)
t=time.time()-start
print ('**************************Train_Time = epoch', t, '  mins: ',divmod(divmod(t, 3600)[1], 60)[0])
#11.55

('Train Finished for file ', '06_159.xml', ' loss_val=', 0.26006278)
('Train Finished for file ', '06_460.xml', ' loss_val=', 0.23167025)
('Train Finished for file ', '06_1396.xml', ' loss_val=', 0.21112649)
('Train Finished for file ', '06_713.xml', ' loss_val=', 0.18972269)
('Train Finished for file ', '06_1260.xml', ' loss_val=', 0.18287097)
('Train Finished for file ', '06_707.xml', ' loss_val=', 0.15664534)
('Train Finished for file ', '06_1326.xml', ' loss_val=', 0.16367835)
('Train Finished for file ', '06_613.xml', ' loss_val=', 0.14326651)
('Train Finished for file ', '06_1641.xml', ' loss_val=', 0.10886615)
('Train Finished for file ', '06_35.xml', ' loss_val=', 0.094205946)
('Train Finished for file ', '06_1078.xml', ' loss_val=', 0.10957616)
('Train Finished for file ', '06_1320.xml', ' loss_val=', 0.0882813)
('Train Finished for file ', '06_1158.xml', ' loss_val=', 0.0833614)
('Train Finished for file ', '06_417.xml', ' loss_val=', 0.09157497)
('Train Finished for file ', 

('Train Finished for file ', '06_1470.xml', ' loss_val=', 0.019205004)
('Train Finished for file ', '06_1540.xml', ' loss_val=', 0.014230607)
('Train Finished for file ', '06_672.xml', ' loss_val=', 0.006509091)
('Train Finished for file ', '06_849.xml', ' loss_val=', 0.03732893)
('Train Finished for file ', '06_1458.xml', ' loss_val=', 0.065866075)
('Train Finished for file ', '06_892.xml', ' loss_val=', 0.07149222)
('Train Finished for file ', '06_606.xml', ' loss_val=', 0.052117907)
('Train Finished for file ', '06_1367.xml', ' loss_val=', 0.09523001)
('Train Finished for file ', '06_489.xml', ' loss_val=', 0.0817044)
('Train Finished for file ', '06_70.xml', ' loss_val=', 0.030122556)
('Train Finished for file ', '06_1492.xml', ' loss_val=', 0.06672121)
('Train Finished for file ', '06_76.xml', ' loss_val=', 0.044689786)
('Train Finished for file ', '06_1112.xml', ' loss_val=', 0.09341366)
('Train Finished for file ', '06_315.xml', ' loss_val=', 0.041604295)
('Train Finished for fi

('Train Finished for file ', '06_1436.xml', ' loss_val=', 0.062461223)
('Train Finished for file ', '06_1274.xml', ' loss_val=', 0.056937005)
('Train Finished for file ', '06_228.xml', ' loss_val=', 0.020803638)
('Train Finished for file ', '06_1541.xml', ' loss_val=', 0.020658907)
('Train Finished for file ', '06_77.xml', ' loss_val=', 0.082517184)
('Train Finished for file ', '06_1474.xml', ' loss_val=', 0.028866146)
('Train Finished for file ', '06_22.xml', ' loss_val=', 0.045481212)
('Train Finished for file ', '06_172.xml', ' loss_val=', 0.07021225)
('Train Finished for file ', '06_425.xml', ' loss_val=', 0.027169371)
('Train Finished for file ', '06_902.xml', ' loss_val=', 0.045406174)
('Train Finished for file ', '06_955.xml', ' loss_val=', 0.10401512)
('Train Finished for file ', '06_160.xml', ' loss_val=', 0.04769112)
('Train Finished for file ', '06_1289.xml', ' loss_val=', 0.053879246)
('Train Finished for file ', '06_786.xml', ' loss_val=', 0.14166439)
('Train Finished for 

('Train Finished for file ', '06_1433.xml', ' loss_val=', 0.09167688)
('Train Finished for file ', '06_495.xml', ' loss_val=', 0.034439966)
('Train Finished for file ', '06_1414.xml', ' loss_val=', 0.037666105)
('Train Finished for file ', '06_736.xml', ' loss_val=', 0.02082304)
('Train Finished for file ', '06_813.xml', ' loss_val=', 0.07338535)
('Train Finished for file ', '06_884.xml', ' loss_val=', 0.0279776)
('Train Finished for file ', '06_1071.xml', ' loss_val=', 0.04644954)
('Train Finished for file ', '06_1708.xml', ' loss_val=', 0.062721044)
('Train Finished for file ', '06_1279.xml', ' loss_val=', 0.025422089)
('Train Finished for file ', '06_1550.xml', ' loss_val=', 0.005103463)
('Train Finished for file ', '06_96.xml', ' loss_val=', 0.08430126)
('Train Finished for file ', '06_432.xml', ' loss_val=', 0.05030544)
('Train Finished for file ', '06_1735.xml', ' loss_val=', 0.06417719)
('Train Finished for file ', '06_838.xml', ' loss_val=', 0.09134432)
('Train Finished for fil

('Train Finished for file ', '06_1820.xml', ' loss_val=', 0.013419232)
('Train Finished for file ', '06_174.xml', ' loss_val=', 0.036860004)
('Train Finished for file ', '06_643.xml', ' loss_val=', 0.06878321)
('Train Finished for file ', '06_457.xml', ' loss_val=', 0.033128183)
('Train Finished for file ', '06_1811.xml', ' loss_val=', 0.05153956)
('Train Finished for file ', '06_389.xml', ' loss_val=', 0.078505404)
('Train Finished for file ', '06_903.xml', ' loss_val=', 0.05668359)
('Train Finished for file ', '06_1156.xml', ' loss_val=', 0.043690648)
('Train Finished for file ', '06_1688.xml', ' loss_val=', 0.06755042)
('Train Finished for file ', '06_1390.xml', ' loss_val=', 0.029187912)
('Train Finished for file ', '06_705.xml', ' loss_val=', 0.121129155)
('Train Finished for file ', '06_1075.xml', ' loss_val=', 0.07588528)
('Train Finished for file ', '06_342.xml', ' loss_val=', 0.033656143)
('Train Finished for file ', '06_918.xml', ' loss_val=', 0.080831505)
('Train Finished fo

...

('Train Finished for file ', '08_1576.xml', ' loss_val=', 0.063466236)
('Train Finished for file ', '08_1758.xml', ' loss_val=', 0.106350645)
('Train Finished for file ', '08_1972.xml', ' loss_val=', 0.033404607)
('Train Finished for file ', '08_1679.xml', ' loss_val=', 0.03438638)
('Train Finished for file ', '08_1857.xml', ' loss_val=', 0.037791073)
('Train Finished for file ', '08_49.xml', ' loss_val=', 0.06419653)
('Train Finished for file ', '08_502.xml', ' loss_val=', 0.052983582)
('Train Finished for file ', '08_369.xml', ' loss_val=', 0.06631034)
('Train Finished for file ', '08_1477.xml', ' loss_val=', 0.053617053)
('Train Finished for file ', '08_594.xml', ' loss_val=', 0.06298931)
('Train Finished for file ', '08_1348.xml', ' loss_val=', 0.04906779)
('Train Finished for file ', '08_1545.xml', ' loss_val=', 0.042461332)
('Train Finished for file ', '08_317.xml', ' loss_val=', 0.032825973)
('Train Finished for file ', '08_1085.xml', ' loss_val=', 0.043175172)
('Train Finished 

('Train Finished for file ', '08_1008.xml', ' loss_val=', 0.090170294)
('Train Finished for file ', '08_370.xml', ' loss_val=', 0.032517277)
('Train Finished for file ', '08_781.xml', ' loss_val=', 0.06671561)
('Train Finished for file ', '08_1867.xml', ' loss_val=', 0.032223906)
('Train Finished for file ', '08_1544.xml', ' loss_val=', 0.08062539)
('Train Finished for file ', '08_1338.xml', ' loss_val=', 0.0569884)
('Train Finished for file ', '08_298.xml', ' loss_val=', 0.045958616)
('Train Finished for file ', '08_572.xml', ' loss_val=', 0.033856362)
('Train Finished for file ', '08_269.xml', ' loss_val=', 0.03203562)
('Train Finished for file ', '08_1133.xml', ' loss_val=', 0.04236088)
('Train Finished for file ', '08_1283.xml', ' loss_val=', 0.046188578)
('Train Finished for file ', '08_726.xml', ' loss_val=', 0.044292066)
('Train Finished for file ', '08_425.xml', ' loss_val=', 0.03587975)
('Train Finished for file ', '08_735.xml', ' loss_val=', 0.040556375)
('Train Finished for 

('Train Finished for file ', '08_62.xml', ' loss_val=', 0.04239859)
('Train Finished for file ', '08_442.xml', ' loss_val=', 0.051517036)
('Train Finished for file ', '08_977.xml', ' loss_val=', 0.045039985)
('Train Finished for file ', '08_1592.xml', ' loss_val=', 0.05222038)
('Train Finished for file ', '08_549.xml', ' loss_val=', 0.022240328)
('Train Finished for file ', '08_248.xml', ' loss_val=', 0.052262004)
('Train Finished for file ', '08_402.xml', ' loss_val=', 0.02317058)
('Train Finished for file ', '08_1570.xml', ' loss_val=', 0.0754421)
('Train Finished for file ', '08_895.xml', ' loss_val=', 0.05692327)
('Train Finished for file ', '08_1344.xml', ' loss_val=', 0.052273832)
('Train Finished for file ', '08_215.xml', ' loss_val=', 0.15846235)
('Train Finished for file ', '08_1543.xml', ' loss_val=', 0.03795845)
('Train Finished for file ', '08_1412.xml', ' loss_val=', 0.044019654)
('Train Finished for file ', '08_319.xml', ' loss_val=', 0.027206028)
('Train Finished for fil

('Train Finished for file ', '08_1556.xml', ' loss_val=', 0.05487602)
('Train Finished for file ', '08_1805.xml', ' loss_val=', 0.060002875)
('Train Finished for file ', '08_148.xml', ' loss_val=', 0.08903783)
('Train Finished for file ', '08_147.xml', ' loss_val=', 0.0380338)
('Train Finished for file ', '08_858.xml', ' loss_val=', 0.05206204)
('Train Finished for file ', '08_1461.xml', ' loss_val=', 0.037752457)
('Train Finished for file ', '08_1871.xml', ' loss_val=', 0.047543235)
('Train Finished for file ', '08_1987.xml', ' loss_val=', 0.035761736)
('Train Finished for file ', '08_350.xml', ' loss_val=', 0.03564078)
('Train Finished for file ', '08_256.xml', ' loss_val=', 0.03074359)
('Train Finished for file ', '08_25.xml', ' loss_val=', 0.030176383)
('Train Finished for file ', '08_320.xml', ' loss_val=', 0.091300674)
('Train Finished for file ', '08_1081.xml', ' loss_val=', 0.015463689)
('Train Finished for file ', '08_1958.xml', ' loss_val=', 0.019502465)
('Train Finished for 

('Train Finished for file ', '08_1454.xml', ' loss_val=', 0.06602726)
('Train Finished for file ', '08_1340.xml', ' loss_val=', 0.04889748)
('Train Finished for file ', '08_374.xml', ' loss_val=', 0.02246817)
('Train Finished for file ', '08_1028.xml', ' loss_val=', 0.022457499)
('Train Finished for file ', '08_509.xml', ' loss_val=', 0.025068378)
('Train Finished for file ', '08_35.xml', ' loss_val=', 0.056023274)
('Train Finished for file ', '08_1568.xml', ' loss_val=', 0.053028725)
('Train Finished for file ', '08_1908.xml', ' loss_val=', 0.061366383)
('Train Finished for file ', '08_1674.xml', ' loss_val=', 0.032560073)
('Train Finished for file ', '08_1675.xml', ' loss_val=', 0.06373478)
('Train Finished for file ', '08_543.xml', ' loss_val=', 0.049654588)
('Train Finished for file ', '08_600.xml', ' loss_val=', 0.045814313)
('Train Finished for file ', '08_1527.xml', ' loss_val=', 0.026554823)
('Train Finished for file ', '08_273.xml', ' loss_val=', 0.031419642)
('Train Finished 

In [ ]:
## Saving the model

saver = tf.train.Saver()
saver.save(sess, './outputmodl_test.model')


In [ ]:
## Loading from saved model

# with tf.Session(graph=graph) as new_sess:
#     saver = tf.train.import_meta_graph('./outputmodl_test.model' + '.meta')
#     saver.restore(new_sess, './outputmodl_test.model')

#     mp = {}
#     test_files = files_2009
#     for file in test_files:
#         print('Testing for ', file)
#         score = model.test(new_sess, file)
#         mp[file] = scores
#         break

#     print (mp)


In [66]:
import math
def get_precision(result):
    mean_Res=0.0
    count=1;
    for v in result:
        if not  math.isnan(v):
            mean_Res += v
            count += 1
        else:
            print(v);
    print(mean_Res/count)

def get_key_phrases(phrases):
    res_map = {}
    for phrase in phrases.keys():
        if 'unk' not in phrase:
            res_map[phrase] = phrases[phrase];
    return res_map
            
####Test data
def predict_output(file):
    score = model.test(sess, file)
    test_scores[file] = score
    #print(test_scores);
    #print(test_scores[file].keys())
    
    
    #get_precision(test_scores[file].values())
    return test_scores[file];


predictions = get_key_phrases(predict_output(files_2008[20]))

In [68]:
top_k =  sorted(predictions.values(), reverse = True)[:20]
res = set([])
for k in top_k:
    res.add(predictions.keys()[predictions.values().index(k)])

print (list(res))


['the respondent has filed and', 'order is the fourth affidavit', '13 of the affidavit are', 'to exclude evidence if its', 'it has not filed its', 'i did not give leave', 'but rather were a simpler', 'submitted that this evidence has', 'leave for this affidavit to', 'that this would have required', 'served after the trial had', 'submitted further that this would', 'time before it had been', 'trademark from the food channel', 'deal with issues raised in', 'to the recipes were made', 'the delegate as are in']


In [ ]:
## Running on Test data 

test_files = files_2009
for file in test_files[:10]:
    print('Testing for ', file)
    score = model.test(sess, file)
    test_scores[file] = score
    
print ('***************** DONE *******************')
#print (test_scores)

In [ ]:
sentences, catchphrases = get_statements('09_773.xml');
file_data_frame, catch_phrases = fp.get_dataframe(pd.DataFrame(sentences),pd.DataFrame(catchphrases))
print (catch_phrases)

In [ ]:
set(catch_phrases).intersection(set(test_scores['09_773.xml'].keys()))

In [ ]:
print np.mean(np.array([0.28357031968247953, 0.10526315789473684, 0.3362814007975298, 0.29883349096061285, 0.3347726313633985, 0.34169855681952455, 0.22180451127819548, 0.29621533198318833, 0.3179064851391663, 0.31414265767115607, 0.2857142857142857, 0.320257480762185, 0.3365564042527644, 0.34003879892037786, 0.2643540669856459, 0.2727272727272727, 0.36, 0.31226204086987114, 0.28248198481404463, 0.2922894189854435, 0.35263347763347763, 0.35714285714285715, 0.27367149758454107, 0.2884315882512955, 0.30303304174019063, 0.3097958577370342, 0.22308488612836436, 0.2948317962231006, 0.2651563706625438, 0.25345642525928547, 0.3113800613800614, 0.3132765022925206, 0.31789671104597866, 0.28068712896299103, 0.19166666666666665, 0.23884680134680136, 0.3041553371920851, 0.27534950185995205, 0.2939242686386916, 0.32256552478875533, 0.3045454545454545, 0.3204360754568297, 0.30668430335097, 0.36227036054622264, 0.3216042527651913, 0.3225806451612903, 0.2916666666666667, 0.29012760848314845, 0.3150542909471481, 0.34615384615384615, 0.30020512820512824, 0.25711437685121896, 0.32957393483709274, 0.30318304640746313, 0.3561189993238674, 0.32723112128146453, 0.23076923076923078, 0.29460345915550845, 0.3078537334998301, 0.34509803921568627, 0.2668192918192918, 0.2263888888888889, 0.3039607006759274, 0.30483810876216894, 0.3448275862068966, 0.26813639691782015, 0.3333333333333333, 0.2916666666666667, 0.2615614948672657, 0.2735436131859478, 0.32154503788694966, 0.26423423423423426, 0.32299916457811195, 0.27780680087942605, 0.25, 0.3108315295815296, 0.28697146962233167, 0.36293436293436293, 0.3420833333333333, 0.30220230454209956, 0.3162393162393162, 0.22252747252747251, 0.2571428571428571, 0.27388016857277286, 0.28061798630431695, 0.3082344874155219, 0.32433563377041635, 0.30534911590467145, 0.31573366149979054, 0.3485058045402873, 0.29618585558161986, 0.2888668412298888, 0.23248053392658508, 0.30425219941348974, 0.28011360620056275, 0.34034785195495165, 0.27011494252873564, 0.2902939808758307, 0.2874936147918372, 0.3542871421432141, 0.3256006493506493, 0.28549294130629305, 0.3077051668221137, 0.2426470588235294, 0.31619136136317066, 0.27974005780220207, 0.2987514192895425, 0.2690496488450453, 0.32142857142857145, 0.2613987461595916, 0.32058810107030233, 0.2918595981195691, 0.28839869281045755, 0.2989306599712628, 0.30305602716468594, 0.3282873988756342, 0.3476190476190476, 0.33695652173913043, 0.27856263194933145, 0.3120347394540943, 0.3124354341524804, 0.28866000844669953, 0.24085538540176768, 0.3139350007419131, 0.27951201737605497, 0.25501373626373625, 0.3156475468975469, 0.3235236606719983, 0.26596459096459096, 0.27774742093327387, 0.3474137931034483, 0.30359372962889386, 0.2980520017590956, 0.2792688521855189, 0.3015593760448923, 0.33638825570714115, 0.2941784766235444, 0.2888996043403718, 0.3089133089133089, 0.2867269736842105, 0.3241944107913007, 0.28852002164502166, 0.2721115944120626, 0.29819295101553167, 0.3021492240028635, 0.25, 0.31907121201914984, 0.2424183006535948, 0.3094564609811434, 0.318854800593931, 0.30346283783783784, 0.29464679484912965, 0.3173974718092365, 0.40909090909090906, 0.32652852145355893, 0.29721880577401516, 0.2742237128769637, 0.29914599402191133, 0.30404135338345867, 0.3121794871794872, 0.269538936856207, 0.285070242809271, 0.3248148148148148, 0.2760292110615139, 0.29613485497892605, 0.28370910698496904, 0.29075343124724684, 0.29147857401885513, 0.21896551724137933, 0.25837288845455814, 0.35034515013734957, 0.29179721879950715, 0.375, 0.2949584575963252, 0.3087600834794885, 0.2933644669989208, 0.3083801943176943, 0.2636477557530189, 0.32631063644255415, 0.29616295442908347, 0.27419610924018234, 0.29814814814814816, 0.3144580200501253, 0.31235502985963287, 0.27439954878859113, 0.2960259391670022, 0.26062271062271064, 0.30386257344373163, 0.2604977996831544, 0.2769989274658205, 0.26282416327416, 0.2966394716394717, 0.30071678321678325, 0.3130451448642087, 0.34706168831168827, 0.25, 0.3086872104354269, 0.3192282353490352, 0.24988497814584767, 0.3138461538461539, 0.2914700411966485, 0.32463834273259984, 0.3068181818181818, 0.2864437108590334, 0.3333333333333333, 0.2673725743527385, 0.2938271604938271, 0.26491463451947495, 0.2895225177615232, 0.38666666666666666, 0.27881566794610274, 0.2658774250440917, 0.3062596006144393, 0.28613916390354766, 0.3072993827160494, 0.34147431830621483, 0.29166666666666663, 0.3213026556776557, 0.291407967032967, 0.3149650216190782, 0.31371147841262237, 0.26211610671288094, 0.2676190476190476, 0.28012265512265516, 0.2388888888888889, 0.316436673309528, 0.3285849952516619, 0.3506854256854257, 0.3221080060307827, 0.25044118605408927, 0.261003861003861, 0.2754940711462451, 0.2587129057695419, 0.3116883116883117, 0.28967248476041035, 0.2905343617204424, 0.3279569892473118, 0.30997831390620767, 0.27586206896551724, 0.25813838938838934, 0.26381346885732854, 0.32179955102368896, 0.2866099034612512, 0.3256783104359316, 0.36, 0.3086285327772667, 0.3258172063742823, 0.24285714285714285, 0.3429292929292929, 0.2992346677222989, 0.31591397849462366, 0.28297698470112265, 0.2821687370600414, 0.270977207725365, 0.32527056277056277, 0.28869047619047616, 0.3274017782135806, 0.3277757506112079, 0.3214893935636506, 0.3234991595111535, 0.24069593787335722, 0.2873324329731893, 0.2837807353936386, 0.2992063492063492, 0.31407368229808164, 0.29847562705133923, 0.3095060528998707, 0.29436738162326026, 0.3071908703638881, 0.342479674796748, 0.28670667666091004, 0.31017841480026354, 0.3257887949268575, 0.3039975071225071, 0.31602254636534644, 0.317847359419501, 0.3488505526868698, 0.26062821194400143, 0.29780564263322884, 0.27924790650718273, 0.2871664691517633, 0.28271604938271605, 0.29472100419251845, 0.3183532416798137, 0.274440766595939, 0.3219156526690067, 0.375, 0.3039080459770115, 0.2277200891429134, 0.19657893598682874, 0.26051319176319176, 0.2868159039619613, 0.2979310344827586, 0.38095238095238093, 0.2692513368983957, 0.2932593572111508, 0.2719386052719386, 0.29601548929199845, 0.33336483336483336, 0.3058608058608059, 0.30354904354904355, 0.28210445468509987, 0.28081223096965224, 0.31848836713368733, 0.31976310308747313, 0.29225172761051205, 0.2870721120731945, 0.2957040695580594, 0.352847471127041, 0.26586845466155806, 0.34615384615384615, 0.29293339626972204, 0.30168538034442566, 0.3579542321477805, 0.34615384615384615, 0.2840481886534518, 0.3, 0.30694906928977395, 0.28533068783068777, 0.2834928229665072, 0.30624840738009035, 0.29801849040536305, 0.3684210526315789, 0.2826391421242849, 0.29507720887031236, 0.2500343781597573, 0.29549890298337506, 0.2692307692307692, 0.30715237164664655, 0.3317588294260873, 0.28818654610077665, 0.31523692472940595, 0.2652644250716183, 0.3057246376811594, 0.30764077514403926, 0.3183693434332776, 0.2823837179965472, 0.27909517383201593, 0.2832643235736542, 0.30891465421419334, 0.3020863290744999, 0.2872177419354839, 0.3245064741369221, 0.315758300838946, 0.3076923076923077, 0.28738359054396706, 0.2888696120894857, 0.29421091041780695, 0.270481891184872, 0.2956259986485298, 0.2858117852875917, 0.2708078958078958, 0.2838679635472214, 0.3755252978267951, 0.3106705804107524, 0.30579990743148205, 0.2970870429751391, 0.2912148796098228, 0.2521164021164021, 0.3106278450481349, 0.3333333333333333, 0.3362903225806452, 0.2971014492753623, 0.3459051724137931, 0.2856263808394842, 0.30952380952380953, 0.24559742158426368, 0.27321414576802505, 0.3043976789800589, 0.3057742544811511, 0.2851082251082251, 0.27176764583455976, 0.30585047441665086, 0.27638613775818854, 0.29385066171793844, 0.27046254671993963, 0.30980392156862746, 0.2834757834757835, 0.30145703041336475, 0.3008203900861616, 0.32195198563619615, 0.33287100074758874, 0.32637892714075556, 0.2812698412698413, 0.2589173883904244, 0.31770833333333337, 0.3013712769655044, 0.2685495790758949, 0.3055683744385117, 0.30126044932079415, 0.2825162167844805, 0.26096179183135704, 0.29895755585410755, 0.28, 0.2994062593090844, 0.24074074074074073, 0.3115591397849462, 0.36567982456140347, 0.2886908108445777, 0.3368891816717903, 0.3166073214507749, 0.23809523809523808, 0.269489341261819, 0.29509577157529493, 0.3412697348297217, 0.3321255823903442, 0.2861921411921412, 0.4337474120082816, 0.1764411027568922, 0.32747659093623205, 0.3062258934865181, 0.32, 0.2903225806451613, 0.30160135916714864, 0.3183981844486046, 0.3116230493273694, 0.28611301127173816, 0.2916925145439639, 0.26218180254557966, 0.30542035433702097, 0.2871982191218184, 0.27827164753160955, 0.27708056478405313, 0.3787751873051329, 0.22944444444444448, 0.2995229026900691, 0.32921469137153736, 0.3371717171717172, 0.4583333333333333, 0.2884765859407085, 0.30797322312662323, 0.32366902428344063, 0.302738943768278, 0.30199834548180704, 0.29587455487210934, 0.31332459716775624, 0.3304818483114177, 0.30392401479358, 0.2965836468745455, 0.2874252803079025, 0.3242846042846043, 0.39744074571660776, 0.3425589574140299, 0.23525798525798525, 0.29777545159671975, 0.3022334147334147, 0.28946527350629736, 0.3264090436712795, 0.2807142857142857, 0.23214285714285715, 0.28365940307214293, 0.3459901707213535, 0.3241385902031063, 0.32987064872122346, 0.3070469324667226, 0.28903700857352776, 0.3021748578055567, 0.27993390360443676, 0.2615820802005012, 0.269984485022079, 0.33904761904761904, 0.3333333333333333, 0.31866752288637173, 0.3564814814814815, 0.2702702702702703, 0.25772885020196845, 0.287902956629131, 0.31288819875776397, 0.2687719298245614, 0.30289746104391413, 0.3160776283412546, 0.3049202727067839, 0.31658403517135447, 0.2917606575243016, 0.3420535168923475, 0.2985078191249299, 0.3113161551092586, 0.3090021512890994, 0.2553886853886854, 0.29598740172190263, 0.2889483499427994, 0.14285714285714285, 0.25, 0.3268633540372671, 0.2676899551123154, 0.30254088561541975, 0.3365427347272362, 0.3613193403298351, 0.2992815139587019, 0.29007747909349735, 0.32216494529575335, 0.3060048403513711, 0.23142857142857143, 0.31074803332867845, 0.313851000146262, 0.2766894899315992, 0.30927856374220136, 0.3413905339366378, 0.27194661965612177, 0.30916309350445104, 0.3351648351648352, 0.29166738478268195, 0.3028824265898768, 0.29220562557952207, 0.3732193732193732, 0.2689940655393498, 0.2857142857142857, 0.3181818181818182, 0.32659977703455967, 0.3254004060455673, 0.28648176008108095, 0.37194901872321223, 0.3254214496946789, 0.3127262622681509, 0.29366945555199836, 0.2994496880309238, 0.23333333333333334, 0.32416666666666666, 0.2439311594202898, 0.2913961038961039, 0.3511904761904762, 0.2916699346405229, 0.3084142384108188, 0.3797953964194373, 0.2980392156862745, 0.30921855921855923, 0.2960533755219003, 0.33244979013193704, 0.2842540475253724, 0.3329351650156633, 0.30978433470690847, 0.33516598344184556, 0.3264772092411844, 0.2993442736257986, 0.2710786505657273, 0.2904154453067497, 0.31026349038706025, 0.30760670526295525, 0.3060428849902534, 0.2722797202851827, 0.2894872707147375, 0.31411558511113574, 0.3014788421416519, 0.3074778634175549, 0.335731809285758, 0.2972569204330503, 0.2651612903225806, 0.26844810795948587, 0.2727272727272727, 0.3569023569023569, 0.3061930763935776, 0.34615384615384615, 0.28001360985113205, 0.2871017871017871, 0.3145582416978257, 0.3100026455026455, 0.3033279309587562, 0.2965207631874298, 0.3214274575327207, 0.3333333333333333, 0.32789236972704716, 0.3075727504867522, 0.30303030303030304, 0.28798111772249707, 0.3032585303504537, 0.30140797876305125, 0.32390655321689804, 0.30304333016936896, 0.3083343955924601, 0.31540116133866136, 0.27285558302039037, 0.3636020647905198, 0.29779903489580906, 0.30148431278866056, 0.34615384615384615, 0.2879076479076479, 0.2872642736806823, 0.33322190574312816, 0.32610264132003264, 0.30075013309613746, 0.26822393100778297, 0.31605780850602583, 0.3343756027414198, 0.3338830584707646, 0.3231093583584005, 0.2938767127181761, 0.2833389607967319, 0.32307692307692304, 0.28597232330264, 0.3652482985816319, 0.3007518796992481, 0.2826224457215837, 0.2861282483157741, 0.36363636363636365, 0.28862219043036685, 0.29919003725600424, 0.11764705882352941, 0.30666666666666664, 0.27241116913050295, 0.3147656290513433, 0.31955925403933594, 0.34325396825396826, 0.3212487105806071, 0.31902682839140006, 0.30803159217879467, 0.29215353611905337, 0.3190743338008415, 0.3031770501404648, 0.32386363636363635, 0.3084715028742885, 0.17647058823529413, 0.34656770515840984, 0.2886813490109854, 0.2880840912076975, 0.29914655508295, 0.2860835180700248, 0.2955702465523894, 0.34814814814814815, 0.3202890902890903, 0.28977146042363433, 0.300544181458778, 0.2830010302845617, 0.31685074236593025, 0.2857142857142857, 0.3082988655651904, 0.29284992784992786, 0.2878602683993262, 0.27237891737891734, 0.2847024177493713, 0.2897465437788018, 0.317789631466839, 0.38095238095238093, 0.23809523809523808, 0.3761528326745718, 0.32686202686202687, 0.29285616344439874, 0.2833694083694083, 0.30943126084594585, 0.2722666117822455, 0.3141404572439055, 0.3126770424664939, 0.3647500647500647, 0.27195198451460867, 0.3149451115797663, 0.32231755154168945, 0.39571150097465885, 0.26090627638615255, 0.30610670194003525, 0.3217261904761905, 0.2962962962962963, 0.31185033389873745, 0.25209692451071763, 0.27404915687128406, 0.2956788831302179, 0.28026745711725554, 0.3187828322854528, 0.31792114695340495, 0.30648805835040654, 0.30290913126694, 0.2843682944751393, 0.3018645303971391, 0.32214064393267855, 0.2702361673414305, 0.30056980056980054, 0.2950576578517332, 0.274110060690943, 0.3145367091317635, 0.264012679050273, 0.2564102564102564, 0.3664656706734352, 0.3232597223976534, 0.3333333333333333, 0.32495726495726496, 0.31507831942614556, 0.3096816976127321, 0.3031601850632413, 0.29166666666666663, 0.30572818760449944, 0.32493355454708106, 0.26855410290192894, 0.26860568833748905, 0.2852090438297335, 0.31403966403966405, 0.2694872734668916, 0.3512584394528291, 0.2928087238432066, 0.2923626966935321, 0.30218241625362363, 0.375, 0.29727109006106534, 0.2643426748689906, 0.27083333333333337, 0.295026355691437, 0.29789083200847905, 0.3280532212885154, 0.34615384615384615, 0.2941047769875826, 0.2753846153846154, 0.3448934725405314, 0.31032863047269316, 0.2715617715617715, 0.30733695652173915, 0.3167586099423406, 0.32328357364541577, 0.3333333333333333, 0.13333333333333333, 0.30230414746543777, 0.28690057635460925]))